<a href="https://colab.research.google.com/github/shusank8/SEQUENCEModels/blob/main/LSTMFROMScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print('SIMPLE LONG SHORT TERM MEMORY; BY SHUSANKET BASYAL')

SIMPLE LONG SHORT TERM MEMORY; BY SHUSANKET BASYAL


In [3]:
# LOADING THE DATASET
# DATASET IS THE SHORT JOKES FROM KAGGLE
import kagglehub
path = kagglehub.dataset_download("abhinavmoudgil95/short-jokes")

print("Path to dataset files:", path)

100%|██████████| 9.82M/9.82M [00:00<00:00, 48.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/abhinavmoudgil95/short-jokes/versions/1


In [4]:
# IMPORTING THE NECESSARY LIBARIES

import os
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F

In [5]:
# LOOKING WHERE THE FILES HAS BEEN DOWNLOADED
os.listdir(path)

['shortjokes.csv']

In [6]:
# LOADING THE FILE INTO DF
df = pd.read_csv(path+"/shortjokes.csv")
# GETTING ALL THE VALUES IN JOKE COLUMN => RETURNS A LIST
text = df['Joke'].values
# JOINING ALL THE STR VAL IN THE LIST TO GET A SINGLE STR
text = "".join(text)
# GETTING THE UNIQUE CHAR PRESENT IN THE DATASET AND CREATING A VARIABLE VOCAB_SIZE THAT STORES THE LEN OF THE UNIQUE ELEMENTS
char = sorted(list(set(text)))
vocab_size = len(char)
# SIMPLE ENCODER, DECODER
# CREATING A HASMAP THAT MAPS STRING TO ID AND VICE VERSA
stringtoid = {sti:i for i,sti in enumerate(char)}
idtostring = {i:sti for i, sti in enumerate(char)}
# USING THE CREATED HASMAP TO CREATER ENCODER AND DECODER
encode = lambda x : [stringtoid[i] for i in x]
decode = lambda x: "".join([idtostring[i] for i in x])
# ENCODING THE TEXT
text = torch.tensor(encode(text), dtype=torch.long)
# CREATING TRAIN AND VAL SIZE
n = int(0.8*len(text))
train = text[0:n]
val = text[n:]

In [7]:
# This function creates batches of data for training or validation.
# It selects random starting points, extracts sequences of a given length (block_size), and prepares input (x) and target (y) tensors for a model.

def generate_batch(split, batch_size, block_size):
  data = train if split =='train' else val
  idx = torch.randint(0, len(data)-block_size, (batch_size, ))
  x = torch.stack([data[i:i+block_size] for i in idx])
  y = torch.stack([data[i+1:i+1+block_size] for i in idx])
  return x,y


In [8]:
# This function estimates the model's loss on the validation set by running 64 mini-batches through it.
# It calculates cross-entropy loss for each batch and returns the average loss, temporarily switching the model to evaluation mode for accurate assessment.

def estimate_loss(model, vocab_size, batch_size, block_size):

  model.eval()

  losses = torch.zeros(64)
  for _ in range(64):
    x,y = generate_batch('val', batch_size, block_size)
    x = x.to('cuda')
    y = y.to('cuda')
    logits = model(x)
    logits = logits.reshape(-1, vocab_size)
    y = y.view(-1)
    loss = F.cross_entropy(logits, y)
    losses[_] = loss.item()
  model.train()
  return losses.mean()


In [9]:
embdim = 64
block_size = 64
hidim = 64
# outdim = 32
batch_size = 128
vocab_size

97

In [10]:
class LSTMFROMScratch(nn.Module):

  def __init__(self):
    super().__init__()

    self.embeddings = nn.Embedding(vocab_size, embdim)

    # forget gate
    self.forget_gate_x = nn.Linear(embdim, hidim, bias = False)
    self.forget_gate_hid = nn.Linear(hidim, hidim, bias = False)

    # input gate
    self.input_gate_x = nn.Linear(embdim, hidim, bias= False)
    self.input_gate_hid = nn.Linear(hidim, hidim, bias= False)

    # candidate gate
    self.candidate_gate_x = nn.Linear(embdim, hidim, bias = False)
    self.candidate_gate_hid = nn.Linear(hidim, hidim, bias = False)

    # output gate
    self.outputgate_x = nn.Linear(embdim,hidim, bias = False)
    self.outputgate_hid = nn.Linear(hidim,hidim, bias = False)

    # self.input_to_hidden = nn.Linear(embdim, hidim,bias = False)

    # self.hidden_to_hidden = nn.Linear(hidim, hidim, bias = False)

    self.out = nn.Linear(hidim, vocab_size, bias = False)

  def forward(self, x, h=None, c = None):
    x = self.embeddings(x)
    # shape of x => (B,T,C)
    x = x.transpose(0,1)
    T,B,C = x.shape
    if h is None:
      h = torch.zeros(B, hidim, device = 'cuda')
      c = torch.zeros(B, hidim, device = 'cuda')
    res = []
    for _ in range(T):

      xi = x[_]

      # a = self.input_to_hidden(xi)

      # b = self.hidden_to_hidden(h)

      # z = a+b

      fg = torch.sigmoid(self.forget_gate_x(xi)+self.forget_gate_hid(h))

      ig = torch.sigmoid(self.input_gate_x(xi)+self.input_gate_hid(h))

      cg = torch.tanh(self.candidate_gate_x(xi)+self.candidate_gate_hid(h))

      og = torch.sigmoid(self.outputgate_x(xi)+self.outputgate_hid(h))

      c = c*fg + ig*cg

      h = torch.tanh(c)*og

      ot = self.out(h)

      res.append(ot)

    res = torch.stack(res)

    res = res.transpose(0,1)

    return res




In [11]:
# class LSTMPY(nn.Module):

#   def __init__(self):
#     super().__init__()
#     self.embeddings = nn.Embedding(vocab_size, embdim)
#     self.lstm = nn.LSTM(embdim, hidim, 1, True, True)
#     self.out = nn.Linear(hidim, vocab_size, bias=False)
#   def forward(self, x):
#     x = self.embeddings(x)
#     out,hid = self.lstm(x)
#     return self.out(out)

In [12]:
# model = LSTMPY()
# for name, p in model.named_parameters():
#   print(name, p.size())

In [13]:
model = LSTMFROMScratch()
for name, param in model.named_parameters():
  if param.dim()>=2:
    torch.nn.init.xavier_normal_(param)
model = model.to("cuda")

optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [25]:
epoches = 10000
for _ in range(epoches):

  x,y = generate_batch('train', batch_size, block_size)
  x = x.to("cuda")
  y = y.to("cuda")
  logits = model(x)
  logits = logits.reshape(-1, vocab_size)
  y = y.view(-1)
  loss = F.cross_entropy(logits, y)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  if _%200==0:

    l = estimate_loss(model, vocab_size, batch_size, block_size)
    print("step:", _ , "loss=>", l.item())

step: 0 loss=> 1.7409148216247559
step: 200 loss=> 1.7359461784362793
step: 400 loss=> 1.7355518341064453
step: 600 loss=> 1.7298846244812012
step: 800 loss=> 1.7306774854660034
step: 1000 loss=> 1.7333598136901855
step: 1200 loss=> 1.738206148147583
step: 1400 loss=> 1.7317755222320557
step: 1600 loss=> 1.7363221645355225
step: 1800 loss=> 1.7335338592529297
step: 2000 loss=> 1.7332932949066162
step: 2200 loss=> 1.7348014116287231
step: 2400 loss=> 1.7307429313659668
step: 2600 loss=> 1.7306863069534302
step: 2800 loss=> 1.7286218404769897
step: 3000 loss=> 1.7274980545043945
step: 3200 loss=> 1.7333952188491821
step: 3400 loss=> 1.7272214889526367
step: 3600 loss=> 1.7223597764968872
step: 3800 loss=> 1.7258373498916626
step: 4000 loss=> 1.7239192724227905
step: 4200 loss=> 1.727766752243042
step: 4400 loss=> 1.7290436029434204
step: 4600 loss=> 1.7228200435638428
step: 4800 loss=> 1.726190447807312
step: 5000 loss=> 1.722676396369934
step: 5200 loss=> 1.724047303199768
step: 5400 lo

In [40]:

# This function generates tokens using the trained model.
# Starting from a given input, it predicts the next token, samples from the probability distribution, appends it to the sequence,
# and continues for max_tok steps without updating gradients.

def generatetok(model, start, max_tok):
  with torch.no_grad():
    for _ in range(max_tok):
      start2 = start[:, -block_size:]
      B,T = start.shape
      logits = model(start2)

      # logits = logits.reshape(-1, vocab_size)
      prob = logits[:,-1,:]
      prob = F.softmax(prob, dim=-1)
      lo = torch.multinomial(prob, num_samples=1)
      start = torch.cat([start, lo], dim=1)
  return start


In [38]:
start = torch.tensor([2,2,2,2,2,2], device='cuda',dtype=torch.long).reshape(6,1)

In [36]:
# INITIALIZING THE START AS 0
# start = torch.zeros([3,1], device='cuda',dtype=torch.long)
# GENERATING FROM THE MODEL
out = generatetok(model, start, block_size*2)
out.shape
# output
res = []
for _ in range(start.shape[0]):
  o = out[_]
  res.append(decode(o.tolist()))
for x in res:
  print(x)
  print("--------------\n")


 roper addy drniver outsistion to the papt doesn't pict that kilking a baby dick paperionsery it holless to this demusions whould
--------------

 are like a cons.If theside? It's the dobs painition at Gres driving pingli, fire Block me.More?, malither Man" stolen was greese
--------------

 this fromber man's potoes ...What do you teat. Lint have hegs? A bear? Shoulder.All, Lands who days F2] he was a brown. What's a
--------------

 rans is? Even learss It's GRYHO-hay Let and make the other mom I againsish for a horse.My wife". It's always relarritally.A feet
--------------

 celled breakbalre commonist in a good Futed Houred my legut I marrier, all nuts some chicken son asked 7 "Your snow at holding a
--------------

 joke a pee, visor.If you, first drivenage realinging: Bentants ask a nuile hip in her all have an artermin: I vacubach on Hothin
--------------



In [39]:
# INITIALIZING THE START AS 0
# start = torch.zeros([3,1], device='cuda',dtype=torch.long)
# GENERATING FROM THE MODEL
out = generatetok(model, start, block_size*2)
out.shape
# output
res = []
for _ in range(start.shape[0]):
  o = out[_]
  res.append(decode(o.tolist()))
for x in res:
  print(x)
  print("--------------\n")


 went cotchature? That they want to sweer 218 we's winchTproke into her rifie untody pook subbed any aniewhere wowl he calls bein
--------------

 brings, both now Amm.O,? just a black is die g: I wikk, I reading president,  could be went pandare."Most in boof just doinger h
--------------

 difference betweerahe Ludban's small! Nothing bring in this with a baby "I have to meet ountia know, I cann'te gay on me.The bos
--------------

 a here you burn?" That wains a joke room Fred. Clivinisian.You, he's not I pade" and from the un) Hod with CraraurioWhy.If he's 
--------------

 numsed, "It's a prickles Konat? The pawrs] BRWY The wor corts.The Bact ovl David, what you call a parts? Packles when I through 
--------------

 spraces makes thank a minolide and Phedice skirings, what, it's me lave me? Me: Gooding that's bed, wows a botA THERAIESS are ev
--------------

